In [21]:
import pandas as pd
import numpy as np
from tika import parser
import os
import glob
import json
import jsonlines
import random
import re
from pprint import pprint
from random import sample

In [2]:
bik_df = pd.read_csv('Bik_v2.tsv', sep='\t', encoding='unicode-escape')
bik_df.columns

Index(['Unnamed: 0', 'Authors', 'Title', 'Citation', 'DOI', 'Year', 'Month',
       '0', '1', '2', '3', 'FINDINGS', 'Reported', 'Correction Date',
       'Retraction', 'Correction', 'No Action', 'SUM \nCompleted', 'URL',
       'Home Site', 'Lab Size', 'Pub Rate', 'Other Journals',
       'First Author Affiliation', 'First Author Career Duration',
       'First Author Degree', 'First Author Degree Area', 'university_name',
       'world_rank_x', 'country_x', 'national_rank', 'quality_of_education',
       'alumni_employment', 'quality_of_faculty', 'publications', 'influence',
       'citations_x', 'broad_impact', 'patents', 'score', 'year_x',
       'world_rank_y', 'country_y', 'teaching', 'international', 'research',
       'citations_y', 'income', 'total_score', 'num_students',
       'student_staff_ratio', 'international_students', 'female_male_ratio',
       'year_y', 'city_ascii', 'state_id', 'state_name', 'county_fips',
       'county_name', 'lat', 'lng', 'population', 'density',

In [3]:
bik_df["Title"]

0      Chronic Morphine Treatment Attenuates Cell Gro...
1      Katanin Localization Requires Triplet Microtub...
2      Curcumin Modulates the Inflammatory Response a...
3      miR-221 Promotes Tumorigenesis in Human Triple...
4      HIV-1 Tat Protein Increases Microglial Outward...
                             ...                        
209    Drosophila Mgr, a Prefoldin subunit cooperatin...
210    The costimulatory immunogen LPS induces the B-...
211    Shox2 is a molecular determinant of depot-spec...
212    LRP-6 is a coreceptor for multiple fibrogenic ...
213    Interplay of mevalonate and Hippo pathways reg...
Name: Title, Length: 214, dtype: object

In [4]:
pdf_files = glob.glob('./PDFs'+"/*.pdf")
pdf_files = sorted(pdf_files, key=lambda x: int(re.match('\D*(\d+)', x).group(1)))

print(len(pdf_files))

214


In [5]:
with open('./april2019_set_mini.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    #pprint(result)

In [6]:
stopword1_1='Abstract'
stopword1_2='ABSTRACT'
stopword1_3='Summary'
stopword1_4='SUMMARY'
stopword1_5='a b s t r a c t'
stopword2_1='Introduction'
stopword2_2='1. Introduction'
stopword2_3='1.Introduction'

count_ab =0
count_int=0

for i in range(len(pdf_files)):
    
    parsed_file = parser.from_file(pdf_files[i])
    extracted_text = parsed_file["content"].strip()
    
    if stopword1_1 in extracted_text:
        pass
    elif stopword1_2 in extracted_text:
        pass
    elif stopword1_3 in extracted_text:
        pass
    elif stopword1_4 in extracted_text:
        pass
    elif stopword1_5 in extracted_text:
        pass
    else:
        print('Abstract not found in text', i)
        count_ab += 1
    
    if stopword2_1 in extracted_text:
        pass
    elif stopword2_2 in extracted_text:
        pass
    elif stopword2_3 in extracted_text:
        pass
    else:
        print('Introduction not found in text', i)
        count_int += 1
        
##
##53-55: 'introduction' appears in the Rerference section..
##183-187: no introduction #no stopword applicable
##188-193: no abstract #No stopwords applicable
### PNAS: 208~213: no abstract, introduction #No stopwords applicable

Introduction not found in text 48
Introduction not found in text 49
Introduction not found in text 50
Abstract not found in text 51
Introduction not found in text 51
Abstract not found in text 52
Introduction not found in text 52
Abstract not found in text 53
Abstract not found in text 54
Abstract not found in text 55
Introduction not found in text 55
Abstract not found in text 56
Introduction not found in text 56
Abstract not found in text 57
Introduction not found in text 57
Abstract not found in text 58
Introduction not found in text 58
Abstract not found in text 59
Introduction not found in text 59
Abstract not found in text 60
Introduction not found in text 60
Abstract not found in text 61
Introduction not found in text 61
Abstract not found in text 62
Introduction not found in text 62
Abstract not found in text 63
Introduction not found in text 63
Abstract not found in text 64
Introduction not found in text 64
Abstract not found in text 65
Introduction not found in text 65
Abstra

In [7]:
print('No stopword for intro:', count_int)
print('No stopword for abstract:', count_ab)

No stopword for intro: 67
No stopword for abstract: 78


> Using stopwords we can extract summary and text separately.. but only half of them? is it worth a work?

In [8]:
master_file = []

for i in range(len(pdf_files)):

    parsed_file = parser.from_file(pdf_files[i])
    extracted_text = parsed_file["content"].strip()
    
    newinput ={}
    
    newinput["title"] = bik_df['Title'][i]
    newinput['warc_date'] = '20220328'
    newinput["text"]= extracted_text #+ ' [CLS]'
    newinput["inst_index"] = str(random.randint(9999,100000)) #radom 5 figure number
    newinput["domain"] = bik_df['Citation'][i]
    newinput["url"] = bik_df['URL'][i]
    newinput["summary"] = "" #Blank until we figure out the abstract stuff
    newinput['authors'] = [bik_df['Authors'][i]]
    newinput['publish_date'] =  '03-28-' + str(bik_df['Year'][i])  #Needs actual published date in form of 'M-D-2022' but try with today + publushed year
    newinput["split"] = "gen"
    newinput["status"] = "succcess"
    
    master_file.append(newinput)


In [13]:
with open('Bik_text.jsonl','w', encoding='utf-8') as f:
    for item in master_file:
        f.write(json.dumps(item) + "\n")